In [64]:
from typing import Optional
import random 

import requests

from api_connection import NotionAPIConnector

connector = NotionAPIConnector()
raw_response:dict = connector.main()
headers = connector.headers

In [51]:
class ResultFetcher:
    
    PAGE_COL_NAME = "Pages"
    # DESC_COL_NAME = "Description"
    
    """ 
    Fetch the results of the raw json response 
    """
    
    def __init__(self, raw_response:dict):
        self.raw_response = raw_response
        
    
    @staticmethod
    def create_initial_dict(raw_response:dict) -> dict:
        """ 
        Return a dict with the main subject pages as key and the page id as title
        """
        # On itére dans la database directement là (première couche)
        results = raw_response["results"]
        
        initial_dict = {}
        
        page:dict
        for page in results:
            page_id = page['id']
            page_name = page['properties']['Pages']['title'][0]['plain_text']
            initial_dict[page_name] = page_id
            
        return initial_dict
    
    @staticmethod
    def get_response_results(response_json:dict) -> list:
        return response_json.get("results", [])
    
    
""" 
Maintenant qu'on a les pages principales il faut ; 
- Qu'on itère dans chaque main page 
- Qu'on détecte les pages enfants
- Qu'on récupére les subpages des pages enfants 
- Si jamais une subpage ne contient que des pages et pas de texte, il ne faut pas l'ajouter dans le système 
- Il faut garder le tag (le nom principale qui nous vient du premier dict) de chaque sous page

"""
    
        
fetcher = ResultFetcher(raw_response=raw_response)

fetcher.create_initial_dict(raw_response)

{'Ton per': 'fcbca7da-3517-4163-9362-21525d5784f1',
 'ta mere': '80497c93-1232-4710-b14d-b45a27563932',
 'Page test': 'e54dd41c-3ac2-47bb-b721-8cbb744990de'}

In [52]:
len(raw_response["results"])

3

In [53]:
block_id = "fcbca7da-3517-4163-9362-21525d5784f1"
block_id

'fcbca7da-3517-4163-9362-21525d5784f1'

In [54]:
# Get subpages
url = f'https://api.notion.com/v1/blocks/{block_id}/children'


response = requests.get(url, headers=headers)

print(len(response.json()["results"])) # 2 subpages

1


In [55]:
response.json()

{'object': 'list',
 'results': [{'object': 'block',
   'id': '6c7ba276-b6ed-4d39-ade2-8e29cf0cd321',
   'parent': {'type': 'page_id',
    'page_id': 'fcbca7da-3517-4163-9362-21525d5784f1'},
   'created_time': '2024-08-23T12:20:00.000Z',
   'last_edited_time': '2024-08-23T12:20:00.000Z',
   'created_by': {'object': 'user',
    'id': '095667f4-633c-4a3c-9ae2-b0da56bd7e27'},
   'last_edited_by': {'object': 'user',
    'id': '095667f4-633c-4a3c-9ae2-b0da56bd7e27'},
   'has_children': False,
   'archived': False,
   'in_trash': False,
   'type': 'paragraph',
   'paragraph': {'rich_text': [{'type': 'text',
      'text': {'content': 'Salut toi', 'link': None},
      'annotations': {'bold': False,
       'italic': False,
       'strikethrough': False,
       'underline': False,
       'code': False,
       'color': 'default'},
      'plain_text': 'Salut toi',
      'href': None}],
    'color': 'default'}}],
 'next_cursor': None,
 'has_more': False,
 'type': 'block',
 'block': {},
 'request_id'

In [56]:
response.json()["results"]

[{'object': 'block',
  'id': '6c7ba276-b6ed-4d39-ade2-8e29cf0cd321',
  'parent': {'type': 'page_id',
   'page_id': 'fcbca7da-3517-4163-9362-21525d5784f1'},
  'created_time': '2024-08-23T12:20:00.000Z',
  'last_edited_time': '2024-08-23T12:20:00.000Z',
  'created_by': {'object': 'user',
   'id': '095667f4-633c-4a3c-9ae2-b0da56bd7e27'},
  'last_edited_by': {'object': 'user',
   'id': '095667f4-633c-4a3c-9ae2-b0da56bd7e27'},
  'has_children': False,
  'archived': False,
  'in_trash': False,
  'type': 'paragraph',
  'paragraph': {'rich_text': [{'type': 'text',
     'text': {'content': 'Salut toi', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': 'Salut toi',
     'href': None}],
   'color': 'default'}}]

In [57]:
# id de la subpage, le but c'est de récup son URL
subpage_id = response.json().get("results")[0]["id"]

url = f'https://api.notion.com/v1/blocks/{subpage_id}/children'

response = requests.get(url, headers=headers)

In [58]:
url = f'https://api.notion.com/v1/pages/{subpage_id}'

response = requests.get(url, headers=headers)

response.json()

{'object': 'error',
 'status': 404,
 'code': 'object_not_found',
 'message': 'Could not find page with ID: 6c7ba276-b6ed-4d39-ade2-8e29cf0cd321. Make sure the relevant pages and databases are shared with your integration.',
 'request_id': '8783ba35-5b12-4880-ad8f-5810afb9703f'}

In [59]:
response.json()

{'object': 'error',
 'status': 404,
 'code': 'object_not_found',
 'message': 'Could not find page with ID: 6c7ba276-b6ed-4d39-ade2-8e29cf0cd321. Make sure the relevant pages and databases are shared with your integration.',
 'request_id': '8783ba35-5b12-4880-ad8f-5810afb9703f'}